In [81]:
# Script to compute TOU pricing for each time period in a dataset and return a modified dataset.
# Input: CSV file of daily consumption with time/date data as one column
# Output: CSV file of daily consumption with TOU pricing data added

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

# Plot formatting defaults
plt.rc('ytick', direction='out')
plt.rc('grid', color='w', linestyle='solid')
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams.update({'font.size': 14})
plt.rc('xtick', direction='out')
plt.rc('patch', edgecolor='#E6E6E6')
plt.rc('lines', linewidth=2)


In [82]:
# TODO: Holidays?

SUMMER_MONTHS = [6, 7, 8, 9]  # June 1 through Sept 31
WINTER_MONTHS = [1, 2, 3, 4, 5, 10, 11, 12]  # Oct 1 through May 31th
ON_PEAK = [16, 17, 18, 19, 20]  # 4pm - 9pm, same for all days
SUMMER_OFF_PEAK = [
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    21,
    22,
    23,
]  # all other hours
SUPER_OFF_PEAK = [

]
WINTER_OFF_PEAK = [
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    21,
    22,
    23,
]  # all other hours

# all in $/kWh
SUM_ON_PEAK_TOU = 0.49271
SUM_OFF_PEAK_TOU = 0.36965 

WIN_ON_PEAK_TOU = 0.35608
WIN_OFF_PEAK_TOU = 0.31728

In [83]:
ca_ids = [3687, 6377, 7062, 8574, 9213, 203, 1450, 1524, 2606, 3864, 7114,
       1731, 4495, 8342, 3938, 5938, 8061, 9775, 4934, 8733, 9612, 9836,
       6547]

In [84]:
df_all = pd.read_csv('15minute_data_california.csv')
df_all = df_all[['dataid','local_15min', 'grid', 'solar']]

In [85]:
for idx in ca_ids:

    # idx = ca_ids[1]

    df = df_all.loc[df_all.dataid == idx]
    map1 = map(lambda s: s[:-3], df.local_15min.to_numpy())
    df["datet"]= pd.to_datetime(list(map1))
    df.reset_index(inplace=True, drop=True)
    df.drop("local_15min", inplace=True, axis=1)
    df.drop("dataid", inplace=True, axis=1)
    if df.solar.isnull().values.any():
        df.solar = np.zeros((len(df),))
        
    df = df.set_index("datet")
    yr = df.index.strftime("%Y")[0]
    if (df.index[0].month == 1) & (df.index[-1].month == 12): # If dataset actually begins and ends with january and march (e.g. not the case for 9836)
        df = df.reindex(pd.date_range(start="01-01-" + yr, end="12-31-" + yr + " 23:45:00", freq="15T"))
        df = df.interpolate()

    df = df.assign(tariff="")

    # Summer TOU pricing, weekdays:
    df.loc[
        df.index.month.isin(SUMMER_MONTHS)
        & df.index.hour.isin(ON_PEAK)
        & df.index.weekday.isin([1, 2, 3, 4, 5]),
        "tariff",
    ] = SUM_ON_PEAK_TOU
    df.loc[
        df.index.month.isin(SUMMER_MONTHS)
        & df.index.hour.isin(SUMMER_OFF_PEAK)
        & df.index.weekday.isin([1, 2, 3, 4, 5]),
        "tariff",
    ] = SUM_OFF_PEAK_TOU

    # Winter TOU pricing, weekdays:
    df.loc[
        df.index.month.isin(WINTER_MONTHS)
        & df.index.hour.isin(ON_PEAK)
        & df.index.weekday.isin([1, 2, 3, 4, 5]),
        "tariff",
    ] = WIN_ON_PEAK_TOU
    df.loc[
        df.index.month.isin(WINTER_MONTHS)
        & df.index.hour.isin(WINTER_OFF_PEAK)
        & df.index.weekday.isin([1, 2, 3, 4, 5]),
        "tariff",
    ] = WIN_OFF_PEAK_TOU

    # Summer TOU pricing, weekends:
    df.loc[
        df.index.month.isin(SUMMER_MONTHS)
        & df.index.weekday.isin([0, 6]),
        "tariff",
    ] = SUM_OFF_PEAK_TOU

    # Winter TOU pricing, weekends:
    df.loc[
        df.index.month.isin(WINTER_MONTHS)
        & df.index.weekday.isin([0, 6]),
        "tariff",
    ] = WIN_OFF_PEAK_TOU

    # Save to CSV
    df.to_csv("load_tariff_" + str(idx) + ".csv")

In [ ]:
# Plot!
fig = plt.figure(figsize=(8, 6), dpi=150)
ax = plt.gca()
# df.plot(kind='line', x='dt', y='grid', ax=ax, xlabel='Date', ylabel='Power, kW')
# df.plot(kind='line', x='dt', y='solar', color='red', ax=ax, xlabel='Date', ylabel='Power, kW')
df.plot(
    kind="line",
    y="load",
    color="green",
    ax=ax,
    xlabel="Date",
    ylabel="Power [kW]",
)
fig.savefig("load_data.png")

# Plot tariff rate!
fig = plt.figure(figsize=(8, 6), dpi=150)
ax = plt.gca()
# df.plot(kind='line', x='dt', y='grid', ax=ax, xlabel='Date', ylabel='Power, kW')
# df.plot(kind='line', x='dt', y='solar', color='red', ax=ax, xlabel='Date', ylabel='Power, kW')
df.plot(
    kind="line",
    y="tariff",
    color="blue",
    ax=ax,
    xlabel="Date",
    ylabel="Cost [$/kWh]",
)
fig.savefig("tariff_data.png")


# Plot PV!
fig = plt.figure(figsize=(8, 6), dpi=150)
ax = plt.gca()
# df.plot(kind='line', x='dt', y='grid', ax=ax, xlabel='Date', ylabel='Power, kW')
# df.plot(kind='line', x='dt', y='solar', color='red', ax=ax, xlabel='Date', ylabel='Power, kW')
df.plot(
    kind="line",
    y="solar",
    color="red",
    ax=ax,
    xlabel="Date",
    ylabel="Power [kW]",
)
fig.savefig("solar_data.png")
